In [1]:
import copy
import qcodes
from qcodes import Parameter

In [2]:
class Powersource(qcodes.Instrument):
    def __init__(self, name): 
        super().__init__(name)
        self._voltage = 0
        
        self.add_parameter(
            name="voltage", 
            set_cmd=self._set_voltage,
            get_cmd=self._get_voltage,
            unit="mV"  # Stupid thing works in mV instead of Volt
        )
        
    def _set_voltage(self, value): 
        self._voltage = value
        
    def _get_voltage(self):
        return self._voltage

In [3]:

class LinearMapping: 
    def __init__(self, conversion_factor): 
        self._conversion_factor = conversion_factor
    
    def __call__(self, value): 
        return value * self._conversion_factor
    
    def inverse(self, value): 
        return value / self._conversion_factor

class Alias(Parameter):
    def __init__(self, parameter, name, instrument=None, mapping=None, **kwargs):
        
        self._parameter = parameter
        self.name = name

        if not instrument:
            instrument = getattr(parameter, "_instrument", None)
            
        super().__init__(
            name=self.name,
            instrument=instrument,
            metadata=parameter.metadata,
            **kwargs
        )

        if mapping is None:
            self._mapping = LinearMapping(1)
        
        elif isinstance(mapping, float) or isinstance(mapping, int):
            self._mapping = LinearMapping(mapping)
        else:
            self._mapping = mapping
        
        if instrument is not None: 
            instrument.parameters[name] = self

    def set(self, value):
        self._save_val(value)
        self._parameter(self._mapping(value))

    def get(self):
        value = self._mapping.inverse(self._parameter())
        self._save_val(value)
        return value

In [4]:
def alias(parameter, name, instrument=None, mapping=None, **kwargs):
    return Alias(parameter, name, instrument=instrument, mapping=mapping, **kwargs)

In [11]:
# This cell does not work (yet), but maybe this is the way forward 

def alias(parameter, mapping, **kwargs): 
    
    if isinstance(mapping, float) or isinstance(mapping, int): 
        mapping = LinearMapping(mapping)
    
    # The proposal is to add a copy method on the Parameter base class. The copy method shall return a new parameter 
    # of the same type and with the same attributes (including instrument) as the original, except where this is overriden 
    # by the copy arguments. 
    new_parameter = parameter.copy(
        set_cmd=lambda value: parameter(mapping(value)), 
        get_cmd=lambda: mapping.inverse(parameter()), 
        **kwargs
    )
    
    return new_parameter

In [5]:
pwr = Powersource("pwr")

In [6]:
gate = alias(pwr.voltage, name="gate", mapping=1E3, unit="V", label="gate voltage")

In [7]:
gate(3) # Set at 3 V

In [8]:
gate()

3.0

In [9]:
pwr.voltage()

0.003

In [10]:
pwr.snapshot()

{'__class__': '__main__.Powersource',
 'functions': {},
 'name': 'pwr',
 'parameters': {'IDN': {'__class__': 'qcodes.instrument.parameter.StandardParameter',
   'instrument': '__main__.Powersource',
   'instrument_name': 'pwr',
   'label': 'IDN',
   'name': 'IDN',
   'ts': None,
   'unit': '',
   'vals': '<Anything>',
   'value': None},
  'gate': {'__class__': '__main__.Alias',
   'instrument': '__main__.Powersource',
   'instrument_name': 'pwr',
   'label': 'gate voltage',
   'name': 'gate',
   'ts': '2017-11-16 15:38:17',
   'unit': 'V',
   'vals': '<Numbers>',
   'value': 3.0},
  'voltage': {'__class__': 'qcodes.instrument.parameter.StandardParameter',
   'instrument': '__main__.Powersource',
   'instrument_name': 'pwr',
   'label': 'voltage',
   'name': 'voltage',
   'ts': '2017-11-16 15:38:17',
   'unit': 'mV',
   'vals': '<Numbers>',
   'value': 0.003}},
 'submodules': {}}

In [11]:
pwr.voltage.snapshot()

{'__class__': 'qcodes.instrument.parameter.StandardParameter',
 'instrument': '__main__.Powersource',
 'instrument_name': 'pwr',
 'label': 'voltage',
 'name': 'voltage',
 'ts': '2017-11-16 15:38:17',
 'unit': 'mV',
 'vals': '<Numbers>',
 'value': 0.003}

In [12]:
gate.snapshot()

{'__class__': '__main__.Alias',
 'instrument': '__main__.Powersource',
 'instrument_name': 'pwr',
 'label': 'gate voltage',
 'name': 'gate',
 'ts': '2017-11-16 15:38:17',
 'unit': 'V',
 'vals': '<Numbers>',
 'value': 3.0}

In [13]:
gate.metadata

{}

In [14]:
pwr.voltage.metadata

{}

In [15]:


prefix_table = {
    "nano": 1E-9,
    "micro": 1E-6,
    "milli": 1E-3,
    "unit": 1,
    "kilo": 1000,
    "mega": 1E6,
    "giga": 1E9
}



In [19]:
list(combinations(prefix_table, 2))

[('nano', 'micro'),
 ('nano', 'milli'),
 ('nano', 'unit'),
 ('nano', 'kilo'),
 ('nano', 'mega'),
 ('nano', 'giga'),
 ('micro', 'milli'),
 ('micro', 'unit'),
 ('micro', 'kilo'),
 ('micro', 'mega'),
 ('micro', 'giga'),
 ('milli', 'unit'),
 ('milli', 'kilo'),
 ('milli', 'mega'),
 ('milli', 'giga'),
 ('unit', 'kilo'),
 ('unit', 'mega'),
 ('unit', 'giga'),
 ('kilo', 'mega'),
 ('kilo', 'giga'),
 ('mega', 'giga')]

In [21]:
conversion_table = {
    "{} to {}".format(frm, to): prefix_table[frm] / prefix_table[to] for frm, to in combinations(prefix_table, 2)
}

In [22]:
conversion_table.update(
    {"{} to {}".format(to, frm): prefix_table[to] / prefix_table[frm] for frm, to in combinations(prefix_table, 2)}
)

In [23]:
conversion_table["milli to unit"]

0.001

In [3]:
from itertools import combinations

prefix_table = {
    "nano": 1E-9,
    "micro": 1E-6,
    "milli": 1E-3,
    "unit": 1,
    "kilo": 1000,
    "mega": 1E6,
    "giga": 1E9
}

class LinearMapping:

    conversion_table = {
        "{} to {}".format(frm, to): prefix_table[frm] / prefix_table[to] for frm, to in combinations(prefix_table, 2)
    }
    
    conversion_table.update(
        {"{} to {}".format(to, frm): prefix_table[to] / prefix_table[frm] for frm, to in combinations(prefix_table, 2)}
    )
    
    def __init__(self, conversion_factor):

        if isinstance(conversion_factor, str):
            self.conversion_factor = LinearMapping.conversion_table[conversion_factor]
        else:
            self.conversion_factor = conversion_factor

    def __call__(self, value):
        return value / self.conversion_factor

    def inverse(self, value):
        return value * self.conversion_factor

    def __repr__(self):
        return "Linear map: this = original * {}".format(self.conversion_factor)

In [26]:
mapping = LinearMapping("milli to unit")

In [27]:
mapping.conversion_factor

0.001

In [28]:
mapping = LinearMapping("unit to milli")

In [29]:
mapping.conversion_factor

1000.0